In [19]:
import pandas as pd

In [20]:
IN_DIR = "../datasets/LongSpeechSource/TEDLIUM_release-3"
IN_DIR = "/mnt/d/voicedata/spgispeech/spgispeech"
OUT_DIR = '../datasets/LongSpeech'
source_df = pd.read_json(OUT_DIR + "/raw_cuts.jsonl", lines=True)

In [21]:
source_df.head()

,id,start,duration,channel,supervisions,recording,type
0,0018ad922e541b415ae60e175160b976_118-0,0,6.54,0,[{'id': '0018ad922e541b415ae60e175160b976_118'...,"{'id': '0018ad922e541b415ae60e175160b976_118',...",MonoCut
1,0018ad922e541b415ae60e175160b976_120-1,0,6.60,0,[{'id': '0018ad922e541b415ae60e175160b976_120'...,"{'id': '0018ad922e541b415ae60e175160b976_120',...",MonoCut
2,0018ad922e541b415ae60e175160b976_148-2,0,8.94,0,[{'id': '0018ad922e541b415ae60e175160b976_148'...,"{'id': '0018ad922e541b415ae60e175160b976_148',...",MonoCut
3,0018ad922e541b415ae60e175160b976_21-3,0,5.73,0,"[{'id': '0018ad922e541b415ae60e175160b976_21',...","{'id': '0018ad922e541b415ae60e175160b976_21', ...",MonoCut
4,0018ad922e541b415ae60e175160b976_3-4,0,9.18,0,"[{'id': '0018ad922e541b415ae60e175160b976_3', ...","{'id': '0018ad922e541b415ae60e175160b976_3', '...",MonoCut


In [22]:
def prepare_and_group(
        df: pd.DataFrame,
    ):

    df = df[['id', 'duration']].copy()

    parts = df['id'].str.split('-').str[0].str.split('_', expand=True)

    df[['speaker', 'segment_num']] = parts[[0, 1]]
    df['segment_num'] = df['segment_num'].astype(int)
    df['duration'] = df['duration'].astype(float)

    return df

In [23]:


def group_segments(
    df: pd.DataFrame,
    target_sec: int = 600,
    tol_sec: int = 90
):

    df_sorted = df.sort_values(["speaker", "segment_num"]).reset_index(drop=True)

    lower, upper = target_sec - tol_sec, target_sec + tol_sec
    groups, cur_ids, cur_dur, cur_spk = [], [], 0.0, None

    for _, row in df_sorted.iterrows():
        spk, seg_id, dur = row["speaker"], row["id"], float(row["duration"])

        if cur_spk is not None and spk != cur_spk and cur_dur >= lower:
            groups.append(cur_ids)
            cur_ids, cur_dur = [], 0.0

        cur_ids.append(seg_id)
        cur_dur += dur
        cur_spk = spk

        if cur_dur >= upper:
            groups.append(cur_ids)
            cur_ids, cur_dur, cur_spk = [], 0.0, None

    if cur_ids:
        groups.append(cur_ids)

    return groups


In [24]:
processed_df = prepare_and_group(df=source_df)
grouped_segments = group_segments(df=processed_df)
grouped_segments

[['0018ad922e541b415ae60e175160b976_3-4',
  '0018ad922e541b415ae60e175160b976_21-3',
  '0018ad922e541b415ae60e175160b976_34-5',
  '0018ad922e541b415ae60e175160b976_72-6',
  '0018ad922e541b415ae60e175160b976_92-7',
  '0018ad922e541b415ae60e175160b976_118-0',
  '0018ad922e541b415ae60e175160b976_120-1',
  '0018ad922e541b415ae60e175160b976_148-2',
  '002d12258ff802d65f79ae2eef99e4ab_10-8',
  '002d12258ff802d65f79ae2eef99e4ab_16-17',
  '002d12258ff802d65f79ae2eef99e4ab_21-23',
  '002d12258ff802d65f79ae2eef99e4ab_28-33',
  '002d12258ff802d65f79ae2eef99e4ab_39-34',
  '002d12258ff802d65f79ae2eef99e4ab_42-35',
  '002d12258ff802d65f79ae2eef99e4ab_51-36',
  '002d12258ff802d65f79ae2eef99e4ab_56-37',
  '002d12258ff802d65f79ae2eef99e4ab_75-38',
  '002d12258ff802d65f79ae2eef99e4ab_81-39',
  '002d12258ff802d65f79ae2eef99e4ab_83-40',
  '002d12258ff802d65f79ae2eef99e4ab_89-41',
  '002d12258ff802d65f79ae2eef99e4ab_99-42',
  '002d12258ff802d65f79ae2eef99e4ab_102-9',
  '002d12258ff802d65f79ae2eef99e4ab_108